In [ ]:
def test_model(model, test_loader, criterion, device):
    model.eval()
    total_loss = 0.0
    correct = 0
    total = 0
    threshold = 1.0

    with torch.no_grad():
        for (img1, lbp1), (img2, lbp2), label in test_loader:
            img1, lbp1 = img1.to(device), lbp1.to(device)
            img2, lbp2 = img2.to(device), lbp2.to(device)
            label = label.to(device)

            cosine_label = torch.where(label == 1, torch.tensor(-1.0, device=device), torch.tensor(1.0, device=device))

            output1, output2 = model((img1, lbp1), (img2, lbp2))

            loss = criterion(output1, output2, cosine_label)
            total_loss += loss.item()

            cos_sim = F.cosine_similarity(output1, output2)

            predictions = (cos_sim > threshold).float()

            predicted_labels = (predictions == 0).float()

            correct += (predicted_labels == label).sum().item()
            total += label.size(0)

    avg_loss = total_loss / len(test_loader)
    accuracy = correct / total * 100.0
    print(f"\n✅ Test Loss: {avg_loss:.4f} | Test Accuracy: {accuracy:.2f}%")

test_model(model, test_loader, criterion, device)